In [ ]:
!git clone https://github.com/openai/shap-e

Cloning into 'shap-e'...
remote: Enumerating objects: 336, done.
remote: Counting objects: 100% (55/55), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 336 (delta 35), reused 13 (delta 13), pack-reused 281 (from 2)
Receiving objects: 100% (336/336), 11.72 MiB | 14.36 MiB/s, done.
Resolving deltas: 100% (43/43), done.


In [ ]:
%cd shap-e

/content/shap-e


In [ ]:
!pip install -e .

Obtaining file:///content/shap-e
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-install-rwg6ffh4/clip_8b32160c76e7413fa3d6da5accdf4eea
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-install-rwg6ffh4/clip_8b32160c76e7413fa3d6da5accdf4eea
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 120.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.6 MB/s eta 0:00:00
   

In [ ]:
import torch

from shap_e.diffusion.sample import sample_latents
from shap_e.diffusion.gaussian_diffusion import diffusion_from_config
from shap_e.models.download import load_model, load_config
from shap_e.util.notebooks import create_pan_cameras, decode_latent_images, gif_widget
from shap_e.util.image_util import load_image

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

here is loaded both model for image to 3d and text to 3d

In [83]:
xm = load_model('transmitter', device=device)
#model = load_model('text300M', device=device) # this model is for text to 3d
model = load_model('image300M', device=device) # this model is for image to 3d
diffusion = diffusion_from_config(load_config('diffusion'))

/content/shap-e/shap_e/models/download.py:136: FutureWarning:

You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.



In [88]:
# for image to 3d model
batch_size = 1
guidance_scale = 10.0

# To get the best result, you should remove the background and show only the object of interest to the model.
image = load_image("/content/pngtree-big-lion-walking-wild-cat-png-image_7088507.png")

latents = sample_latents(
    batch_size=batch_size,
    model=model,
    diffusion=diffusion,
    guidance_scale=guidance_scale,
    model_kwargs=dict(images=[image] * batch_size),
    progress=True,
    clip_denoised=True,
    use_fp16=True,
    use_karras=True,
    karras_steps=150,
    sigma_min=1e-3,
    sigma_max=160,
    s_churn=0,
)

  0%|          | 0/150 [00:00<?, ?it/s]

In [93]:
# for text to 3d model
# batch_size = 1
# guidance_scale = 10.0
# prompt = "a shark"

# latents = sample_latents(
#     batch_size=batch_size,
#     model=model,
#     diffusion=diffusion,
#     guidance_scale=guidance_scale,
#     model_kwargs=dict(texts=[prompt] * batch_size),
#     progress=True,
#     clip_denoised=True,
#     use_fp16=True,
#     use_karras=True,
#     karras_steps=64,
#     sigma_min=1e-3,
#     sigma_max=160,
#     s_churn=0,
# )

In [90]:
render_mode = 'nerf'
size = 128 # this is the size of the renders; higher values take longer to render.

cameras = create_pan_cameras(size, device)
for i, latent in enumerate(latents):
    images = decode_latent_images(xm, latent, cameras, rendering_mode=render_mode)
    display(gif_widget(images))

HTML(value='<img src="data:image/gif;base64,R0lGODlhgACAAIcAAO/i2ufTxum/mNi/sOm1ieK0htC2p9O1mOWugOCtfuCpetypet…

In [91]:
from shap_e.util.notebooks import decode_latent_mesh

for i, latent in enumerate(latents):
    t = decode_latent_mesh(xm, latent).tri_mesh()
    with open(f'example_mesh_{i}.obj', 'w') as f:
        t.write_obj(f)

/content/shap-e/shap_e/models/stf/renderer.py:286: UserWarning:

exception rendering with PyTorch3D: No module named 'pytorch3d'

/content/shap-e/shap_e/models/stf/renderer.py:287: UserWarning:

falling back on native PyTorch renderer, which does not support full gradients



In [94]:
import trimesh
import numpy as np
# Load the mesh (OBJ + MTL if available)
mesh = trimesh.load('/content/shap-e/example_mesh_0.obj', process=True)
vertex_colors = mesh.visual.vertex_colors[:, :3] / 255.0  # RGB normalized

face_colors = mesh.visual.face_colors[:, :3] / 255.0
vertex_colors = np.zeros_like(mesh.vertices)
for i, face in enumerate(mesh.faces):
    vertex_colors[face] = face_colors[i]

import plotly.graph_objects as go

vertices = mesh.vertices
faces = mesh.faces

fig = go.Figure(data=[go.Mesh3d(
    x=vertices[:, 0],
    y=vertices[:, 1],
    z=vertices[:, 2],
    i=faces[:, 0],
    j=faces[:, 1],
    k=faces[:, 2],
    vertexcolor=vertex_colors,
    opacity=1.0,
    flatshading=True
)])

fig.update_layout(scene=dict(aspectmode='data'))
fig.show()


In [96]:
# to run the code what ever i writen in top of ipynb file do same this
#